In [48]:
import altair as alt
import numpy as np
import pandas as pd
from scipy.interpolate import PchipInterpolator


In [ ]:
treasury = pd.read_csv(
    "https://home.treasury.gov/resource-center/data-chart-center/interest-rates/daily-treasury-rates.csv/all/202207?type=daily_treasury_yield_curve&field_tdr_date_value_month=202207&page&_format=csv"
)

In [79]:
all_treasuries = pd.read_csv(
    "https://home.treasury.gov/system/files/276/yield-curve-rates-1990-2021.csv"
)

In [114]:
for i in all_treasuries.shape[0]:
all_treasuries.iloc[0]
    

,Date,1 Mo,2 Mo,3 Mo,6 Mo,1 Yr,2 Yr,3 Yr,5 Yr,7 Yr,10 Yr,20 Yr,30 Yr
0,2021-12-31,0.06,0.05,0.06,0.19,0.39,0.73,0.97,1.26,1.44,1.52,1.94,1.90
1,2021-12-30,0.06,0.06,0.05,0.19,0.38,0.73,0.98,1.27,1.44,1.52,1.97,1.93
2,2021-12-29,0.01,0.02,0.05,0.19,0.38,0.75,0.99,1.29,1.47,1.55,2.00,1.96
3,2021-12-28,0.03,0.04,0.06,0.20,0.39,0.74,0.99,1.27,1.41,1.49,1.94,1.90
4,2021-12-27,0.04,0.05,0.06,0.21,0.33,0.76,0.98,1.26,1.41,1.48,1.92,1.88
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8003,1990-01-08,NaN,NaN,7.79,7.88,7.81,7.90,7.95,7.92,8.05,8.02,NaN,8.09
8004,1990-01-05,NaN,NaN,7.79,7.85,7.79,7.90,7.94,7.92,8.03,7.99,NaN,8.06
8005,1990-01-04,NaN,NaN,7.84,7.90,7.82,7.92,7.93,7.91,8.02,7.98,NaN,8.04
8006,1990-01-03,NaN,NaN,7.89,7.94,7.85,7.94,7.96,7.92,8.04,7.99,NaN,8.04


In [115]:
abc

'30 Yr'

In [80]:
all_treasuries.Date = pd.to_datetime(all_treasuries.Date)


In [109]:
def interpolate_curve(curve):
    curve = curve.iloc[1:]
    curve.columns = ["tenor", "value"]
    curve.loc[:, "tenor"] = [1, 2, 3, 6, 12, 24, 36, 60, 90, 120, 240, 360]

    curve = curve.dropna()

    g = PchipInterpolator(curve["tenor"], curve["value"], extrapolate=True)
    tenors = np.linspace(0, 360, 61, endpoint=True)
    values = g(tenors)

    return tenors, values


In [110]:
old_treasury = all_treasuries.iloc[100].reset_index()
curve_date = all_treasuries.iloc[100].Date.strftime('%Y-%m-%d')


tenors, values = interpolate_curve(old_treasury)

curve_data = pd.DataFrame(
    {"tenor": np.linspace(0, 360, 61, endpoint=True), "value": values}
)

curve_data["yield"] = curve_data["yield"] / 100.0
curve_data["maturity"] = curve_data["maturity"] / 12


/var/folders/8g/67y55mms29zf6yt6q92_3hsr0000gn/T/ipykernel_57920/4098965440.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  curve.loc[:, "tenor"] = [1, 2, 3, 6, 12, 24, 36, 60, 90, 120, 240, 360]


In [111]:
alt.Chart(curve_data).mark_line().encode(
    x="maturity", y="yield", tooltip=["maturity", "yield"]
)


alt.Chart(...)

In [ ]:
old_treasury = all_treasuries.iloc[100].reset_index()
curve_date = all_treasuries.iloc[100].Date.strftime('%Y-%m-%d')

tenors, values = interpolate_curve(old_treasury)

curve_data = pd.DataFrame(
    {"tenor": np.linspace(0, 360, 61, endpoint=True), "value": values}
)

curve_data["tenor"] = curve_data["tenor"] / 12
curve_data["value"] = curve_data["value"] / 100.0


In [113]:
curve_values = list(
    zip(
        np.tile("us_treasury", curve_data.shape[0]),
        np.tile(curve_date, curve_data.shape[0]),
        curve_data.maturity,
        curve_data["yield"],
    )
)


In [ ]:
for i in curve_values:
    query = """
        INSERT INTO curve
        (curve_name, date, tenor, value)
        VALUES (%s,%s,%s,%s)
        """
    